In [ ]:
! pip install scrapy
! pip install scrapy-splash

In [ ]:
%%writefile my_spider.py
import scrapy
from scrapy_splash import SplashRequest

class MySpider(scrapy.Spider):
  name = 'my_spider'
  allowed_domains = ['eshop.telekom.sk']

  #
  base_url = 'https://eshop.telekom.sk/category/all-devices/list/product_listing'

  #
  brand_filter = 'Apple'
  items_per_page = 12

  def start_requests(self):
    url = f"{self.base_url}?traffId=RP1348&itemPerPage={self.items_per_page}&filter%5B%5D=filter.brand%5B%5D%3D{self.brand_filter}"
    yield SplashRequest(url, callback=self.parse, meta={'current_page': 1}, args={'wait': 10})

  def parse(self, response):
    current_page = response.meta['current_page']

    #
    product_link = response.css('a[href*="/category/all-devices/list/product_listing/product/"]::attr(href)').getall()

    #
    for link in product_link:
      full_link = response.urljoin(link)
      yield SplashRequest(full_link, callback=self.parse_item, args={'wait': 10})

    #
    next_page_link = response.css('a.pagination-next::attr(href)').get()
    if next_page_link:
      next_page = current_page + 1
      next_page_url = f"{self.base_url}?currentPage={next_page}&traffId=RP1348&itemPerPage={self.items_per_page}&filter%5B%5D=filter.brand%5B%5D%3D{self.brand_filter}"
      yield SplashRequest(next_page_url, callback=self.parse, meta={'current_page': next_page}, args={'wait': 10})


  def parse_item(self, response):
    #
    product_name = response.xpath('/html/body/div[1]/div[4]/main/section/section/div/section/article/div[1]/article/div/section[2]/article/section[1]/section[1]/h1').get()
    availability = response.css('div.sc-feUZmu.cvbGwH.dt_typography.variant_small.badgeText::text').get()
    # price = response.css('').get()
    # description = response.css('').get()

    yield {
        'name': product_name,
        'availability': availability,
        # 'price': price,
        # 'description': description,
        'url': response.url
    }

Overwriting my_spider.py


In [ ]:
!scrapy runspider my_spider.py -o products.json

2024-11-17 14:21:02 [scrapy.utils.log] INFO: Scrapy 2.11.2 started (bot: scrapybot)
2024-11-17 14:21:02 [scrapy.utils.log] INFO: Versions: lxml 5.3.0.0, libxml2 2.12.9, cssselect 1.2.0, parsel 1.9.1, w3lib 2.2.1, Twisted 24.10.0, Python 3.10.12 (main, Sep 11 2024, 15:47:36) [GCC 11.4.0], pyOpenSSL 24.2.1 (OpenSSL 3.3.2 3 Sep 2024), cryptography 43.0.3, Platform Linux-6.1.85+-x86_64-with-glibc2.35
2024-11-17 14:21:02 [scrapy.addons] INFO: Enabled addons:
[]
/usr/local/lib/python3.10/dist-packages/scrapy/utils/request.py:254: ScrapyDeprecationWarning: '2.6' is a deprecated value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting.

It is also the default value. In other words, it is normal to get this warning if you have not defined a value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting. This is so for backward compatibility reasons, but it will change in a future version of Scrapy.

See the documentation of the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting for information on

In [ ]:
from google.colab import files
files.download('products.json')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>